# Automating data wrangling

## Introduction

Sometimes we require a "one off" solution to a unique data analysis problem. In this situation, we write code to do a particular analysis on a particular data set. Then, if the analysis is part of a publication, we make the code and data publically available and... we're done.

Often, however, what we require is a reusable solution that operates on data of a given format even though some of the particulars, such as sample size or variable names, might change. In this case, we want our code to be "dynamic" in the sense that it should be able to handle any anticipated changes to the details of the input data.

## Import pandas and look at the data from last time

In [9]:
import pandas as pd

Read in the data from last time.

In [10]:
my_input_data = pd.read_csv('datasets/017DataFile.csv')

Take a peek to remind ourselves of the structure.

In [11]:
my_input_data.head()

,Male Mutant,Female Mutant,Male Wild Type,Female Wild Type
0,10.485451,8.250013,20.127063,25.946384
1,11.747948,8.453839,20.068147,23.464870
2,13.412580,9.706605,21.215148,22.989480
3,12.910095,9.522116,20.706416,25.324376
4,10.367770,8.583212,18.074795,22.607487


Last time, we stacked the reaction time values into a single column using pandas functions. This relied on us knowing and "hard coding" the column names ("Male Mutant", etc.). If we're going to automate things, we want our code to be agnostic about these. One way would be to somehow read this into variables. But since we already know how to "play legos" with numpy arrays, which are just numbers, let's try that.

In [12]:
import numpy as np

Pandas dataframes know how to convert themselves to numpy arrays. They have a `to_numpy()` method. This is going to pull just the numbers out of our dataframe, ignoring the column labels and row indexs.

In [13]:
raw_data = my_input_data.to_numpy() 

Let's take a look!

In [14]:
raw_data

array([[10.48545088,  8.2500131 , 20.12706278, 25.94638414],
       [11.74794775,  8.45383932, 20.06814699, 23.46487013],
       [13.41258004,  9.70660484, 21.21514789, 22.98948034],
       [12.91009526,  9.52211638, 20.70641578, 25.32437595],
       [10.36777045,  8.58321246, 18.07479515, 22.60748688],
       [11.69842177,  9.83500171, 20.36762403, 23.05218737],
       [11.58315277, 10.53209602, 20.15252058, 25.3690367 ],
       [11.44734892,  9.39416641, 19.39247581, 23.37270897],
       [10.85227619,  8.73947266, 18.52434071, 25.21564644],
       [11.28589742, 10.89239399, 20.32502629, 24.99050453]])

So far so good! Now we are going to put the data into the format we want. To automate this, we are going to get the number of observations in each group – the number of rows – and the number of groups – the number of columns – and store them in variables.

In [15]:
obs, grps = raw_data.shape
print("We have ", obs, " observations per group and ", grps, " groups.")

We have  10  observations per group and  4  groups.


Now we'll calculate the total number of observations, which is also how long we want our new data frame to be.

In [16]:
new_length = obs*grps
print("We have ", new_length, " total observations.")

We have  40  total observations.


We could now play legos "by hand", stacking the columns of our numpy array on top of each other to make a new array (and we already know how to do that). But we can also take advantage of the fact that one of the things numpy arrays know how to do – one of the methods they have – is change their shape. So we'll take our `obs` by `cols` array and `numpy.reshape()` into a `new_length` by 1 array.

In [17]:
values_col = np.reshape(raw_data, (new_length, 1))

I called it `values_col` because it will eventually become the values column of our new pandas data frame.

Let's see if that worked:

In [19]:
values_col

array([[10.48545088],
       [ 8.2500131 ],
       [20.12706278],
       [25.94638414],
       [11.74794775],
       [ 8.45383932],
       [20.06814699],
       [23.46487013],
       [13.41258004],
       [ 9.70660484],
       [21.21514789],
       [22.98948034],
       [12.91009526],
       [ 9.52211638],
       [20.70641578],
       [25.32437595],
       [10.36777045],
       [ 8.58321246],
       [18.07479515],
       [22.60748688],
       [11.69842177],
       [ 9.83500171],
       [20.36762403],
       [23.05218737],
       [11.58315277],
       [10.53209602],
       [20.15252058],
       [25.3690367 ],
       [11.44734892],
       [ 9.39416641],
       [19.39247581],
       [23.37270897],
       [10.85227619],
       [ 8.73947266],
       [18.52434071],
       [25.21564644],
       [11.28589742],
       [10.89239399],
       [20.32502629],
       [24.99050453]])

Nice! But let's make sure that worked. What we want is for the columns of the original data to be stacked on top of one another. Is that what we have?

So what happened is that the values got read out *left to right, top to bottom* (or row-wise) and placed into the new array one-by-one. But what we want is for the values to be read *top to bottom, left to right* (or columnwise). We can make this happen with the `order=` argument.

In [20]:
values_col = np.reshape(raw_data, (new_length, 1), order = 'F')

Let's make sure that worked:

In [21]:
values_col

array([[10.48545088],
       [11.74794775],
       [13.41258004],
       [12.91009526],
       [10.36777045],
       [11.69842177],
       [11.58315277],
       [11.44734892],
       [10.85227619],
       [11.28589742],
       [ 8.2500131 ],
       [ 8.45383932],
       [ 9.70660484],
       [ 9.52211638],
       [ 8.58321246],
       [ 9.83500171],
       [10.53209602],
       [ 9.39416641],
       [ 8.73947266],
       [10.89239399],
       [20.12706278],
       [20.06814699],
       [21.21514789],
       [20.70641578],
       [18.07479515],
       [20.36762403],
       [20.15252058],
       [19.39247581],
       [18.52434071],
       [20.32502629],
       [25.94638414],
       [23.46487013],
       [22.98948034],
       [25.32437595],
       [22.60748688],
       [23.05218737],
       [25.3690367 ],
       [23.37270897],
       [25.21564644],
       [24.99050453]])

**Useless trivia**: Two major programming languages of olde are **C** (used mainly by programmers) and **Fortran** (used mainly by scientists). Fortran uses columnwise indexing, whereas C (the language used to write Python) uses row-wise indexing. That's why "F" is used to specify columnwise indexing above: the "F" is for "Fortran".

Minor annoying thing (there is always at least one that pops up in any coding task): `values_col` is a (40x1) 2-dimensional numpy array but, when we go to build our new data frame, we'll need it to be a 40 long (40,) 1-dimensional array. 

This actually comes up so often that `numpy` has a `squeeze()` function to squeeze the dimension of length one into nothingness. It turns (n, 1) things into (n,) things.

Let's check the shape of 

In [22]:
values_col.shape

(40, 1)

In [23]:
values_col = np.squeeze(values_col)

In [24]:
values_col.shape

(40,)

In [25]:
sexes = pd.Series(['male', 'female'])
sexes = sexes.repeat(obs)
sexes = pd.concat([sexes, sexes], ignore_index=True)

In [26]:
strain = pd.Series(['wildtype', 'mutant'])
strain = strain.repeat(2*obs)
strain = strain.reset_index(drop=True)

In [27]:
my_tidy_data = pd.DataFrame(
    {
        "RTs": values_col,
        "sex": sexes,
        "strain": strain
    }    
)

In [28]:
my_tidy_data

,RTs,sex,strain
0,10.485451,male,wildtype
1,11.747948,male,wildtype
2,13.412580,male,wildtype
3,12.910095,male,wildtype
4,10.367770,male,wildtype
5,11.698422,male,wildtype
6,11.583153,male,wildtype
7,11.447349,male,wildtype
8,10.852276,male,wildtype
9,11.285897,male,wildtype


Yay! We win!

**Important point:** Crucially, *the above code doesn't rely on us knowing much about the input data ahead of time*. As long as it's a pandas data frame that contains numerical values, the code will run. It's automatic.

## Look at new data with more observations with same code

We'll make this code self-contained, so it can be run without running anything above. We'll also add comments, so that future-us can read the code more easily without having to wade through the notebook text above.

In [29]:
# import our libraries
import pandas as pd
import numpy as np

my_input_data = pd.read_csv('datasets/018DataFile.csv')  # read the data

raw_data = my_input_data.to_numpy()                      # convert to numpy array

obs, grps = raw_data.shape                               # get the number of rows and columns

Check the size of the new data real quick:

In [30]:
print("We have ", obs, " observations per group and ", grps, " groups.")

We have  20  observations per group and  4  groups.


In [31]:
new_length = obs*grps                                    # compute total number of observations

values_col = np.reshape(raw_data, (new_length, 1), 
                        order = 'F')                     # reshape the array
values_col = np.squeeze(values_col)                      # squeeze to make 1D

# construct the inner grouping variable
sexes = pd.Series(['male', 'female'])                    # define the levels
sexes = sexes.repeat(obs)                                # make one cycle of the levels
sexes = pd.concat([sexes, sexes], ignore_index=True)     # and repeat the cycle, ditching the indexes

# construct the outer grouping variable
strain = pd.Series(['wildtype', 'mutant'])               # define the levels
strain = strain.repeat(2*obs)                            # make the one cycle
strain = strain.reset_index(drop=True)                   # drop the pesky index

# construct the data frame
my_new_tidy_data = pd.DataFrame(
    {
        "RTs": values_col,                               # make a column named RTs and put the values in
        "sex": sexes,                                    # ditto for sex
        "strain": strain                                 # and for genetic strain
    }    
)

In [32]:
my_new_tidy_data

,RTs,sex,strain
0,12.333785,male,wildtype
1,11.675152,male,wildtype
2,12.029059,male,wildtype
3,12.126430,male,wildtype
4,10.307197,male,wildtype
...,...,...,...
75,24.886821,female,mutant
76,24.475663,female,mutant
77,21.935896,female,mutant
78,23.852748,female,mutant


Success!

## Making the code even more functional

In [33]:
def tidyMyData() :
    import pandas as pd
    import numpy as np

    my_input_data = pd.read_csv('datasets/018DataFile.csv')  # read the data

    raw_data = my_input_data.to_numpy()                      # convert to numpy array

    obs, grps = raw_data.shape                               # get the number of rows and columns

    new_length = obs*grps                                    # compute total number of observations

    values_col = np.reshape(raw_data, (new_length, 1), 
                            order = 'F')                     # reshape the array
    values_col = np.squeeze(values_col)                      # squeeze to make 1D

    # construct the inner grouping variable
    sexes = pd.Series(['male', 'female'])                    # define the levels
    sexes = sexes.repeat(obs)                                # make one cycle of the levels
    sexes = pd.concat([sexes, sexes], ignore_index=True)     # and repeat the cycle, ditching the indexes

    # construct the outer grouping variable
    strain = pd.Series(['wildtype', 'mutant'])               # define the levels
    strain = strain.repeat(2*obs)                            # make the one cycle
    strain = strain.reset_index(drop=True)                   # drop the pesky index

    # construct the data frame
    my_new_tidy_data = pd.DataFrame(
        {
            "RTs": values_col,                               # make a column named RTs and put the values in
            "sex": sexes,                                    # ditto for sex
            "strain": strain                                 # and for genetic strain
        }    
    )
    
    return my_new_tidy_data

In [34]:
datFromFun = tidyMyData()

In [35]:
datFromFun

,RTs,sex,strain
0,12.333785,male,wildtype
1,11.675152,male,wildtype
2,12.029059,male,wildtype
3,12.126430,male,wildtype
4,10.307197,male,wildtype
...,...,...,...
75,24.886821,female,mutant
76,24.475663,female,mutant
77,21.935896,female,mutant
78,23.852748,female,mutant
